In [4]:
import torch
import requests
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification

# 모델 이미지 프로세스 로드
model_name = 'google/vit-base-patch16-224'
image_processor = AutoImageProcessor.from_pretrained(
    model_name,
    use_fast = True
)
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    device_map = 'auto'
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [14]:
# 이미지 로드

image_urls = [
        "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg",
        "http://images.cocodataset.org/val2017/000000039769.jpg",
    ]

for idx, url in enumerate(image_urls,1):
    try:
        # 이미지다운로드
        image = Image.open(requests.get(url, stream = True).raw)    # PIL.Image.open(...) → 이미지를 메모리 내 열기/ requests.get(...).raw → 인터넷에서 이미지 파일 스트림을 가져옴
        print(f'이미지 크기: {image.size}')
        # 이미지 전처리 
        inputs = image_processor(image, return_tensors = 'pt').to(model.device) # 이미지의 전처리가 필요한 이유 - 모델에 입력되는 건 이미지가 아닌 텐서
        # 크기 통일 (예: 224×224)
        # 정규화 (0~1)
        # 채널 배치 (C×H×W)
        # 배치 차원 추가 (1×3×224×224)
        print(f"전처리 후 텐서의 크기 : {inputs['pixel_values'].shape}")
        # 추론
        with torch.no_grad():   # 추론(inference)일 때 gradient 계산 비활성화 (속도↑ 메모리↓)/ gradient(기울기)를 계산하지 않는 모드로 바꾼다/ 학습이 아니라 결과만 낼 때는 역전파가 필요없으니 메모리와 속도를 아끼는 것. " 모델 읽기 전용 모드"라고 생각
            outputs = model(**inputs)   # 이 과정에서 이루어 지는 것 -> 시각적 특징을 파악하고 어느 클래스일 가능성이 높은지 계산하는 단계
                                            # 이미지 패치 쪼개기
                                            # 패치를 linear projection
                                            # 여러 층의 Transformer 인코더 통해 feature 추출
                                            # 마지막에 classifier head 적용
                                            # class별 점수(logit) 생성
            logits = outputs.logits # 모델이 각 클래스 별로 준 원시 점수. 여기서 argmax하면 가장 높은 점수를 가진 클래스가 예측 결과가 된다
        # 결과 해석
        predicted_class_id = logits.argmax(dim=-1).item()
        predicted_class_label = model.config.id2label[predicted_class_id]
        confidence = torch.softmax(logits,dim = 1)[0][predicted_class_id].item()
        print('예측결과: ....')
        print(f'클래스 ID: {predicted_class_id}')
        print(f'클래스 라벨: {predicted_class_label}')
        print(f'확률(신뢰도): {confidence}')
        # top5 예측 결과
        probs = torch.softmax(logits,dim = -1)[0]
        top5_probs, top5_indices = torch.topk(probs, 5)
        for i, (prob, idx) in enumerate(zip(top5_probs,top5_indices),1):
            label = model.config.id2label[idx.item()]
            print(f'    {i}. {label} : {prob.item():.2f}')
    except Exception as e:
        print(f'오류발생 : {e}')

이미지 크기: (960, 686)
전처리 후 텐서의 크기 : torch.Size([1, 3, 224, 224])
예측결과: ....
클래스 ID: 287
클래스 라벨: lynx, catamount
확률(신뢰도): 0.44071435928344727
    1. lynx, catamount : 0.44
    2. cougar, puma, catamount, mountain lion, painter, panther, Felis concolor : 0.03
    3. snow leopard, ounce, Panthera uncia : 0.03
    4. Egyptian cat : 0.02
    5. tiger cat : 0.02
이미지 크기: (640, 480)
전처리 후 텐서의 크기 : torch.Size([1, 3, 224, 224])
예측결과: ....
클래스 ID: 285
클래스 라벨: Egyptian cat
확률(신뢰도): 0.9374890327453613
    1. Egyptian cat : 0.94
    2. tabby, tabby cat : 0.04
    3. tiger cat : 0.01
    4. lynx, catamount : 0.00
    5. Siamese cat, Siamese : 0.00
